In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ib_insync import *
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from optopus.ib_adapter import IBBrokerAdapter
from optopus.optopus import Optopus
from optopus.utils import pdo


%matplotlib inline

In [2]:
# port = 7497 TWS paper trading
# port = 7496 TWS live trading
util.startLoop()
opt = Optopus(IBBrokerAdapter(IB(), host='127.0.0.1', port=7497, client=6))
opt.start()

[Initializating managers]
[Connecting to IB broker]
[Adding underlyings]
- Initializing underlyings:	..
- Retriving current data:	..
- Retriving historical data:	..
- Retriving historical IV data:	..
- Computing fields:		..
[Updating portfolio]

[Started]



# Account

In [3]:
acc = opt.account()
print('[Balances]')
print('Net liquidation\t\t {}'.format(acc['net_liquidation']))
print('Equity with loan\t {}'.format(acc['equity_with_loan']))
print('Total cash\t\t {}'.format(acc['cash']))
print('\n[Margin requierements]')
print('Initial margin\t\t {}'.format(acc['initial_margin']))
print('Maintenance margin\t {}'.format(acc['maintenance_margin']))
print('\n[Available for trading]')
print('Funds\t\t\t {}'.format(acc['funds']))
print('Excess Liquidity\t {}'.format(acc['excess_liquidity']))
print('SMA\t\t\t {}'.format(acc['SMA']))
print('Buying power\t\t {}'.format(acc['buying_power']))
print('Day trades left\t\t {}'.format(acc['max_day_trades']))

[Balances]
Net liquidation		 USD 355375.59
Equity with loan	 USD 355462.72
Total cash		 USD 355268.64

[Margin requierements]
Initial margin		 USD 5119.13
Maintenance margin	 USD 5119.13

[Available for trading]
Funds			 USD 350343.59
Excess Liquidity	 USD 350343.59
SMA			 USD 321051.58
Buying power		 USD 1283960.84
Day trades left		 -1


# Portfolio

In [4]:
pf = opt.portfolio()
print('Beta weighted delta\t', pf['BWDelta'])

Beta weighted delta	 nan


## Positions

In [5]:
print(opt.positions())
for p in opt.positions():
    print('{} {}'.format(p['code'],p['strategy']))

[OrderedDict([('code', 'SPY'), ('asset_type', 'OPT'), ('expiration', datetime.date(2018, 9, 21)), ('strike', 283.0), ('right', 'P'), ('ownership', 'SELL'), ('quantity', 1.0), ('average_cost', 95.5124), ('option_price', nan), ('trade_price', 1.08), ('trade_time', datetime.datetime(2018, 8, 27, 18, 59, 21, 793284)), ('underlying_price', nan), ('beta', 1.0), ('delta', nan), ('algorithm', 'foo'), ('strategy', 'Sell naked put'), ('rol', 'no_rol'), ('DTE', 20)])]
SPY Sell naked put


In [6]:
df_pos = pdo(opt.positions())

In [25]:
df_pos['s_exp'] = pd.to_datetime(df_pos['expiration']).dt.strftime('%b %d')
df_pos['s_tra'] = df_pos['trade_time'].dt.strftime('%d/%m')
df_pos['s_str'] = df_pos['strategy'] + ' (' +df_pos['s_tra'] + ')'
df_pos[['code', 's_str', 'quantity', 'ownership', 's_exp', 'DTE', 'strike', 'right']]

,code,s_str,quantity,ownership,s_exp,DTE,strike,right
0,SPY,Sell naked put (27/08),1.0,SELL,Sep 21,20,283.0,P


ERROR:ib_insync.client:Peer closed connection


In [8]:
df_pos['expiration'] = pd.to_datetime(df_pos['expiration'])

In [9]:
df_pos

,code,asset_type,expiration,strike,right,ownership,quantity,average_cost,option_price,trade_price,trade_time,underlying_price,beta,delta,algorithm,strategy,rol,DTE,str_exp
0,SPY,OPT,2018-09-21,283.0,P,SELL,1.0,95.5124,NaN,1.08,2018-08-27 18:59:21.793284,NaN,1.0,NaN,foo,Sell naked put,no_rol,20,Sep 21


In [10]:
type(df_pos)

pandas.core.frame.DataFrame

In [11]:
df_pos['expiration'].dt.strftime('%b %d')

0    Sep 21
Name: expiration, dtype: object